In [1]:
import ee
import datetime
import os
import itertools
import sys
import re

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('..')

In [4]:
from BackGround_modules.Class_2_Classify_Fourier_Img   import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment
from BackGround_modules.Class_4_Create_Sample_Classification_instancs import Make_Sample_Classification

##### Define_Basic_Parameters

In [5]:
# define the year_name for test, here choose these years because they represent
# Landsat8/7/5 respectively
year_name = ['2017_2019','2011_2013', '2008_2010']
percent_value = [1,2,3,5,7,10,20,30,50,70,100]
tree_num = [1] + list(range(10,121,10))

combination_name = ['Landsat',
                    'Fourier',
                    'Landsat_Fourier',
                    'Landsat_Mean',
                    'Fourier_Mean',
                    'Landsat_Fourier_Mean'
                    'Landsat_Fourier_Climate'
                    'Landsat_Fourier_Terrain'
                    'Landsat_Fourier_Terrain_Climate']

#### Calculate the band_combination accuracy using [Invariant_traning_sample] with different percentage reduction

Produce classification_instances

In [6]:
# define the path to point_with_value GEE-Path
path = 'users/Jinzhu_Deakin/North_China_Plain/Sample_with_Landsat_Fourier_Normalized_Climate'

# instantiate the combo_instance dictionary
Combo_instance_with_village = {}

# Create sample_classification instances through [year] --> [Percent] --> [band_combination] --> [Tree]
for year in year_name:
    
    Invarient_sample = ee.FeatureCollection(f'{path}/Invarient_ext_Img_{year}')
    
    # import samples
    Invarient_built      = Invarient_sample.filterMetadata('Built','equals',1)
    Invarient_non_Built  = Invarient_sample.filterMetadata('Built','equals',0)
       
    for pct in percent_value:
        
        # Create percentage_reduced samples, only use non-built points from verified points
        Subset_Invarient_built     = Make_Sample_Classification.Step_1_Subset_sample(Invarient_built,pct)
        Subset_Invarient_non_Built = Make_Sample_Classification.Step_1_Subset_sample(Invarient_non_Built,pct)
        
        # Merge Verified_points with Zone_points
        Sample_merge = Subset_Invarient_built.merge(Subset_Invarient_non_Built)
        
        # Get the band_combo names
        Band_combo   = Make_Sample_Classification.Step_2_Create_Band_Combo(Sample_merge)
        
        for combo in Band_combo:
            Accuracy_instance = Make_Sample_Classification.\
                                Step_3_Create_Classification_Instance(year,Sample_merge,
                                                                      combo,tree_num,
                                                                      pct,classificaiton_func = Classification ) 
            Combo_instance_with_village.update(Accuracy_instance)
        

Percetage coresponded size is 141
Percetage coresponded size is 87
Classification of 2017_2019_Landsat_pct_001_tree_0001 completed!
Classification of 2017_2019_Landsat_pct_001_tree_0010 completed!
Classification of 2017_2019_Landsat_pct_001_tree_0020 completed!
Classification of 2017_2019_Landsat_pct_001_tree_0030 completed!
Classification of 2017_2019_Landsat_pct_001_tree_0040 completed!
Classification of 2017_2019_Landsat_pct_001_tree_0050 completed!
Classification of 2017_2019_Landsat_pct_001_tree_0060 completed!
Classification of 2017_2019_Landsat_pct_001_tree_0070 completed!
Classification of 2017_2019_Landsat_pct_001_tree_0080 completed!
Classification of 2017_2019_Landsat_pct_001_tree_0090 completed!
Classification of 2017_2019_Landsat_pct_001_tree_0100 completed!
Classification of 2017_2019_Landsat_pct_001_tree_0110 completed!
Classification of 2017_2019_Landsat_pct_001_tree_0120 completed!
Classification of 2017_2019_Fourier_pct_001_tree_0001 completed!
Classification of 2017_

Percetage coresponded size is 283
Percetage coresponded size is 174
Classification of 2017_2019_Landsat_pct_002_tree_0001 completed!
Classification of 2017_2019_Landsat_pct_002_tree_0010 completed!
Classification of 2017_2019_Landsat_pct_002_tree_0020 completed!
Classification of 2017_2019_Landsat_pct_002_tree_0030 completed!
Classification of 2017_2019_Landsat_pct_002_tree_0040 completed!
Classification of 2017_2019_Landsat_pct_002_tree_0050 completed!
Classification of 2017_2019_Landsat_pct_002_tree_0060 completed!
Classification of 2017_2019_Landsat_pct_002_tree_0070 completed!
Classification of 2017_2019_Landsat_pct_002_tree_0080 completed!
Classification of 2017_2019_Landsat_pct_002_tree_0090 completed!
Classification of 2017_2019_Landsat_pct_002_tree_0100 completed!
Classification of 2017_2019_Landsat_pct_002_tree_0110 completed!
Classification of 2017_2019_Landsat_pct_002_tree_0120 completed!
Classification of 2017_2019_Fourier_pct_002_tree_0001 completed!
Classification of 2017

Percetage coresponded size is 425
Percetage coresponded size is 261
Classification of 2017_2019_Landsat_pct_003_tree_0001 completed!
Classification of 2017_2019_Landsat_pct_003_tree_0010 completed!
Classification of 2017_2019_Landsat_pct_003_tree_0020 completed!
Classification of 2017_2019_Landsat_pct_003_tree_0030 completed!
Classification of 2017_2019_Landsat_pct_003_tree_0040 completed!
Classification of 2017_2019_Landsat_pct_003_tree_0050 completed!
Classification of 2017_2019_Landsat_pct_003_tree_0060 completed!
Classification of 2017_2019_Landsat_pct_003_tree_0070 completed!
Classification of 2017_2019_Landsat_pct_003_tree_0080 completed!
Classification of 2017_2019_Landsat_pct_003_tree_0090 completed!
Classification of 2017_2019_Landsat_pct_003_tree_0100 completed!
Classification of 2017_2019_Landsat_pct_003_tree_0110 completed!
Classification of 2017_2019_Landsat_pct_003_tree_0120 completed!
Classification of 2017_2019_Fourier_pct_003_tree_0001 completed!
Classification of 2017

Percetage coresponded size is 709
Percetage coresponded size is 435
Classification of 2017_2019_Landsat_pct_005_tree_0001 completed!
Classification of 2017_2019_Landsat_pct_005_tree_0010 completed!
Classification of 2017_2019_Landsat_pct_005_tree_0020 completed!
Classification of 2017_2019_Landsat_pct_005_tree_0030 completed!
Classification of 2017_2019_Landsat_pct_005_tree_0040 completed!
Classification of 2017_2019_Landsat_pct_005_tree_0050 completed!
Classification of 2017_2019_Landsat_pct_005_tree_0060 completed!
Classification of 2017_2019_Landsat_pct_005_tree_0070 completed!
Classification of 2017_2019_Landsat_pct_005_tree_0080 completed!
Classification of 2017_2019_Landsat_pct_005_tree_0090 completed!
Classification of 2017_2019_Landsat_pct_005_tree_0100 completed!
Classification of 2017_2019_Landsat_pct_005_tree_0110 completed!
Classification of 2017_2019_Landsat_pct_005_tree_0120 completed!
Classification of 2017_2019_Fourier_pct_005_tree_0001 completed!
Classification of 2017

Percetage coresponded size is 993
Percetage coresponded size is 609
Classification of 2017_2019_Landsat_pct_007_tree_0001 completed!
Classification of 2017_2019_Landsat_pct_007_tree_0010 completed!
Classification of 2017_2019_Landsat_pct_007_tree_0020 completed!
Classification of 2017_2019_Landsat_pct_007_tree_0030 completed!
Classification of 2017_2019_Landsat_pct_007_tree_0040 completed!
Classification of 2017_2019_Landsat_pct_007_tree_0050 completed!
Classification of 2017_2019_Landsat_pct_007_tree_0060 completed!
Classification of 2017_2019_Landsat_pct_007_tree_0070 completed!
Classification of 2017_2019_Landsat_pct_007_tree_0080 completed!
Classification of 2017_2019_Landsat_pct_007_tree_0090 completed!
Classification of 2017_2019_Landsat_pct_007_tree_0100 completed!
Classification of 2017_2019_Landsat_pct_007_tree_0110 completed!
Classification of 2017_2019_Landsat_pct_007_tree_0120 completed!
Classification of 2017_2019_Fourier_pct_007_tree_0001 completed!
Classification of 2017

...

Percetage coresponded size is 2838
Percetage coresponded size is 1741
Classification of 2008_2010_Landsat_pct_020_tree_0001 completed!
Classification of 2008_2010_Landsat_pct_020_tree_0010 completed!
Classification of 2008_2010_Landsat_pct_020_tree_0020 completed!
Classification of 2008_2010_Landsat_pct_020_tree_0030 completed!
Classification of 2008_2010_Landsat_pct_020_tree_0040 completed!
Classification of 2008_2010_Landsat_pct_020_tree_0050 completed!
Classification of 2008_2010_Landsat_pct_020_tree_0060 completed!
Classification of 2008_2010_Landsat_pct_020_tree_0070 completed!
Classification of 2008_2010_Landsat_pct_020_tree_0080 completed!
Classification of 2008_2010_Landsat_pct_020_tree_0090 completed!
Classification of 2008_2010_Landsat_pct_020_tree_0100 completed!
Classification of 2008_2010_Landsat_pct_020_tree_0110 completed!
Classification of 2008_2010_Landsat_pct_020_tree_0120 completed!
Classification of 2008_2010_Fourier_pct_020_tree_0001 completed!
Classification of 20

Percetage coresponded size is 4257
Percetage coresponded size is 2611
Classification of 2008_2010_Landsat_pct_030_tree_0001 completed!
Classification of 2008_2010_Landsat_pct_030_tree_0010 completed!
Classification of 2008_2010_Landsat_pct_030_tree_0020 completed!
Classification of 2008_2010_Landsat_pct_030_tree_0030 completed!
Classification of 2008_2010_Landsat_pct_030_tree_0040 completed!
Classification of 2008_2010_Landsat_pct_030_tree_0050 completed!
Classification of 2008_2010_Landsat_pct_030_tree_0060 completed!
Classification of 2008_2010_Landsat_pct_030_tree_0070 completed!
Classification of 2008_2010_Landsat_pct_030_tree_0080 completed!
Classification of 2008_2010_Landsat_pct_030_tree_0090 completed!
Classification of 2008_2010_Landsat_pct_030_tree_0100 completed!
Classification of 2008_2010_Landsat_pct_030_tree_0110 completed!
Classification of 2008_2010_Landsat_pct_030_tree_0120 completed!
Classification of 2008_2010_Fourier_pct_030_tree_0001 completed!
Classification of 20

Percetage coresponded size is 7095
Percetage coresponded size is 4353
Classification of 2008_2010_Landsat_pct_050_tree_0001 completed!
Classification of 2008_2010_Landsat_pct_050_tree_0010 completed!
Classification of 2008_2010_Landsat_pct_050_tree_0020 completed!
Classification of 2008_2010_Landsat_pct_050_tree_0030 completed!
Classification of 2008_2010_Landsat_pct_050_tree_0040 completed!
Classification of 2008_2010_Landsat_pct_050_tree_0050 completed!
Classification of 2008_2010_Landsat_pct_050_tree_0060 completed!
Classification of 2008_2010_Landsat_pct_050_tree_0070 completed!
Classification of 2008_2010_Landsat_pct_050_tree_0080 completed!
Classification of 2008_2010_Landsat_pct_050_tree_0090 completed!
Classification of 2008_2010_Landsat_pct_050_tree_0100 completed!
Classification of 2008_2010_Landsat_pct_050_tree_0110 completed!
Classification of 2008_2010_Landsat_pct_050_tree_0120 completed!
Classification of 2008_2010_Fourier_pct_050_tree_0001 completed!
Classification of 20

Percetage coresponded size is 9933
Percetage coresponded size is 6094
Classification of 2008_2010_Landsat_pct_070_tree_0001 completed!
Classification of 2008_2010_Landsat_pct_070_tree_0010 completed!
Classification of 2008_2010_Landsat_pct_070_tree_0020 completed!
Classification of 2008_2010_Landsat_pct_070_tree_0030 completed!
Classification of 2008_2010_Landsat_pct_070_tree_0040 completed!
Classification of 2008_2010_Landsat_pct_070_tree_0050 completed!
Classification of 2008_2010_Landsat_pct_070_tree_0060 completed!
Classification of 2008_2010_Landsat_pct_070_tree_0070 completed!
Classification of 2008_2010_Landsat_pct_070_tree_0080 completed!
Classification of 2008_2010_Landsat_pct_070_tree_0090 completed!
Classification of 2008_2010_Landsat_pct_070_tree_0100 completed!
Classification of 2008_2010_Landsat_pct_070_tree_0110 completed!
Classification of 2008_2010_Landsat_pct_070_tree_0120 completed!
Classification of 2008_2010_Fourier_pct_070_tree_0001 completed!
Classification of 20

Percetage coresponded size is 14190
Percetage coresponded size is 8706
Classification of 2008_2010_Landsat_pct_100_tree_0001 completed!
Classification of 2008_2010_Landsat_pct_100_tree_0010 completed!
Classification of 2008_2010_Landsat_pct_100_tree_0020 completed!
Classification of 2008_2010_Landsat_pct_100_tree_0030 completed!
Classification of 2008_2010_Landsat_pct_100_tree_0040 completed!
Classification of 2008_2010_Landsat_pct_100_tree_0050 completed!
Classification of 2008_2010_Landsat_pct_100_tree_0060 completed!
Classification of 2008_2010_Landsat_pct_100_tree_0070 completed!
Classification of 2008_2010_Landsat_pct_100_tree_0080 completed!
Classification of 2008_2010_Landsat_pct_100_tree_0090 completed!
Classification of 2008_2010_Landsat_pct_100_tree_0100 completed!
Classification of 2008_2010_Landsat_pct_100_tree_0110 completed!
Classification of 2008_2010_Landsat_pct_100_tree_0120 completed!
Classification of 2008_2010_Fourier_pct_100_tree_0001 completed!
Classification of 2

#### Produce classification_instances <br>

In [ ]:
# innitialize the accuracy dict
Combo_acc_with_village = {}

# innitilize operation flag to report the process
Opts_flag  = 0
Opts_tatal = len(year_name) * len(combination_name) * len(percent_value) * len(tree_num)

# unpack the combo_instance through [year] --> [band_combination] --> [Percent] --> [Tree]
for year in year_name:
    for name in combination_name:
        for pct in percent_value:
            for tree in tree_num:

                Opts_flag+=1

                Combo_acc_with_village[(year,name,pct,tree)] = Accuracy_assesment(Combo_instance_with_village[(year,name,pct,tree)])\
                                              .Stp_1_Calculate_Accuracy()
                # print out the process
                print(f'{year}_{name}_pct_{pct:03}_tree_{tree:03} accuracy computation finished! ==>{Opts_flag:04}/{Opts_tatal}')
            print('_________________')

2017_2019_Landsat_pct_001_tree_001 accuracy computation finished! ==>0001/2574
2017_2019_Landsat_pct_001_tree_010 accuracy computation finished! ==>0002/2574
2017_2019_Landsat_pct_001_tree_020 accuracy computation finished! ==>0003/2574
2017_2019_Landsat_pct_001_tree_030 accuracy computation finished! ==>0004/2574
2017_2019_Landsat_pct_001_tree_040 accuracy computation finished! ==>0005/2574
2017_2019_Landsat_pct_001_tree_050 accuracy computation finished! ==>0006/2574
2017_2019_Landsat_pct_001_tree_060 accuracy computation finished! ==>0007/2574
2017_2019_Landsat_pct_001_tree_070 accuracy computation finished! ==>0008/2574
2017_2019_Landsat_pct_001_tree_080 accuracy computation finished! ==>0009/2574
2017_2019_Landsat_pct_001_tree_090 accuracy computation finished! ==>0010/2574
2017_2019_Landsat_pct_001_tree_100 accuracy computation finished! ==>0011/2574
2017_2019_Landsat_pct_001_tree_110 accuracy computation finished! ==>0012/2574
2017_2019_Landsat_pct_001_tree_120 accuracy computat

2017_2019_Landsat_pct_030_tree_120 accuracy computation finished! ==>0104/2574
_________________
2017_2019_Landsat_pct_050_tree_001 accuracy computation finished! ==>0105/2574
2017_2019_Landsat_pct_050_tree_010 accuracy computation finished! ==>0106/2574
2017_2019_Landsat_pct_050_tree_020 accuracy computation finished! ==>0107/2574
2017_2019_Landsat_pct_050_tree_030 accuracy computation finished! ==>0108/2574
2017_2019_Landsat_pct_050_tree_040 accuracy computation finished! ==>0109/2574
2017_2019_Landsat_pct_050_tree_050 accuracy computation finished! ==>0110/2574
2017_2019_Landsat_pct_050_tree_060 accuracy computation finished! ==>0111/2574
2017_2019_Landsat_pct_050_tree_070 accuracy computation finished! ==>0112/2574
2017_2019_Landsat_pct_050_tree_080 accuracy computation finished! ==>0113/2574
2017_2019_Landsat_pct_050_tree_090 accuracy computation finished! ==>0114/2574
2017_2019_Landsat_pct_050_tree_100 accuracy computation finished! ==>0115/2574
2017_2019_Landsat_pct_050_tree_110

2017_2019_Fourier_pct_007_tree_100 accuracy computation finished! ==>0206/2574
2017_2019_Fourier_pct_007_tree_110 accuracy computation finished! ==>0207/2574
2017_2019_Fourier_pct_007_tree_120 accuracy computation finished! ==>0208/2574
_________________
2017_2019_Fourier_pct_010_tree_001 accuracy computation finished! ==>0209/2574
2017_2019_Fourier_pct_010_tree_010 accuracy computation finished! ==>0210/2574
2017_2019_Fourier_pct_010_tree_020 accuracy computation finished! ==>0211/2574
2017_2019_Fourier_pct_010_tree_030 accuracy computation finished! ==>0212/2574
2017_2019_Fourier_pct_010_tree_040 accuracy computation finished! ==>0213/2574
2017_2019_Fourier_pct_010_tree_050 accuracy computation finished! ==>0214/2574
2017_2019_Fourier_pct_010_tree_060 accuracy computation finished! ==>0215/2574
2017_2019_Fourier_pct_010_tree_070 accuracy computation finished! ==>0216/2574
2017_2019_Fourier_pct_010_tree_080 accuracy computation finished! ==>0217/2574
2017_2019_Fourier_pct_010_tree_090

2017_2019_Landsat_Fourier_pct_002_tree_060 accuracy computation finished! ==>0306/2574
2017_2019_Landsat_Fourier_pct_002_tree_070 accuracy computation finished! ==>0307/2574
2017_2019_Landsat_Fourier_pct_002_tree_080 accuracy computation finished! ==>0308/2574
2017_2019_Landsat_Fourier_pct_002_tree_090 accuracy computation finished! ==>0309/2574
2017_2019_Landsat_Fourier_pct_002_tree_100 accuracy computation finished! ==>0310/2574
2017_2019_Landsat_Fourier_pct_002_tree_110 accuracy computation finished! ==>0311/2574
2017_2019_Landsat_Fourier_pct_002_tree_120 accuracy computation finished! ==>0312/2574
_________________
2017_2019_Landsat_Fourier_pct_003_tree_001 accuracy computation finished! ==>0313/2574
2017_2019_Landsat_Fourier_pct_003_tree_010 accuracy computation finished! ==>0314/2574
2017_2019_Landsat_Fourier_pct_003_tree_020 accuracy computation finished! ==>0315/2574
2017_2019_Landsat_Fourier_pct_003_tree_030 accuracy computation finished! ==>0316/2574
2017_2019_Landsat_Fourier

In [ ]:
# convert the accuracy dicionary into a dataframe
Combo_acc_Invarient_df = pd.DataFrame(Combo_acc_with_village).T
Combo_acc_Invarient_df.index.names = ['Year','Combo','Percent','Tree']

# save the dataframe
Combo_acc_Invarient_df.to_csv('./Step_2_result/Classification_ACC_Invarieant_point.csv')

In [ ]:
Combo_acc_Invarient_df

##### Make figures

In [ ]:
# read the combo_acc
Combo_acc_df = pd.read_csv('./Step_2_result/Classification_ACC_with_zone_point.csv')
Combo_acc_df.set_index('Tree')

Combo_acc_Invarient_df = pd.read_csv('./Step_2_result/Classification_ACC_Invarieant_point.csv')
Combo_acc_Invarient_df.set_index('Tree')

In [ ]:
Full_Percent = Combo_acc_Invarient_df[Combo_acc_Invarient_df['Percent'] == 100]
Full_Percent

In [ ]:
Landsat_5 = Full_Percent[Full_Percent['Year'] == '2008_2010']
Landsat_7 = Full_Percent[Full_Percent['Year'] == '2001_2013']
Landsat_8 = Full_Percent[Full_Percent['Year'] == '2017_2019']

In [ ]:
plt.figure(figsize=(15,8))
sns.lineplot(data = Landsat_5,x = 'Tree',y='Overall_ACC',hue='Combo')

In [ ]:
plt.figure(figsize=(10,6))
fig = sns.FacetGrid(data = Full_Percent,col = 'Year',hue='Combo')
fig.map(sns.lineplot,'Tree','Overall_ACC')

In [ ]:
Combo_acc_df

In [ ]:
g = sns.FacetGrid(data = Combo_acc_df,row = 'Combo',col='Year')
g.map(sns.lineplot,'Percent','Recall_built')